
# Möbius Plateau Auto‑Explorer Ω  
Grandma‑friendly: paste your scan rows into **Cell 2** and press **Runtime ▶ Run all**.

What it does automatically in one pass  

1. Reads your fine‑scan table *(bits, k, φ, λ, rel_err)*.  
2. Tags rows into plateaux at three granularities (ε = 1e‑4, 5 e‑6, 1 e‑5).  
3. Applies **every** quick op we defined so far (×9, ÷9, φ‑family, δ Feigenbaum 6 & α, halves, squares, inverses).  
4. Prints the first 60 rows for every (ε, op) combo so you eyeball structure.  
5. Saves two Excel files in *Content*:  

* **plateaus_raw.xlsx** – untouched rows  
* **plateaus_ops.xlsx** – one sheet per (ε, op)

*(φ golden ratio, α fine‑structure, δ Feigenbaum 5≈4.669, α Feigenbaum ≈ 2.5029)*  

No widgets, no parameters – just run.


In [4]:

import pandas as pd, numpy as np
from itertools import product

# ▣ Paste / update your raw rows here ▣
raw_rows = [
    #  bits   k    φ‑micro       λ     rel_err
    (64,  3.6, 0.003942, 0.0, 0.4117525875906678269),
    (64,  3.8, 0.003942, 0.0, 0.4117525875906678269),
    (64,  4.0, 0.003942, 0.0, 0.4117525875906678269),
    (64,  4.2, 0.003942, 0.0, 0.4117525875906678269),
    (128, 3.6, 0.003942, 0.0, 0.4117525875906678271),
    (128, 3.8, 0.003942, 0.0, 0.4117525875906678271),
    (128, 4.0, 0.003942, 0.0, 0.4117525875906678271),
    (128, 4.2, 0.003942, 0.0, 0.4117525875906678271),
]

df = pd.DataFrame(raw_rows, columns=['bits','k','phi','lam','rel_err'])
print('Preview →'); display(df.head())


Preview →


,bits,k,phi,lam,rel_err
0,64,3.6,0.003942,0.0,0.411753
1,64,3.8,0.003942,0.0,0.411753
2,64,4.0,0.003942,0.0,0.411753
3,64,4.2,0.003942,0.0,0.411753
4,128,3.6,0.003942,0.0,0.411753


In [5]:

# === built‑in constants & ops ===
phi = (1+5**0.5)/2                  # 1.618 …
phi_inv = 1/phi
alpha_inv = 137.035999084          # fine‑structure 1/α
alpha = 1/alpha_inv
delta = 4.669201609               # δ Feigenbaum 6
alpha_F = 2.502907875             # α Feigenbaum

# helper inverses / complements
def comp(c): return 1-c

consts = {
    '9':9, '3':3, '6':6,
    'phi':phi,'phi½':phi**0.5,'phi²':phi**2,'phi⁻¹':phi_inv,'φ':phi,
    'alpha':alpha,'alphaInv':alpha_inv,'α':alpha,'α⁻¹':alpha_inv,
    'delta':delta,'deltaInv':1/delta,'δ':delta,'δ⁻¹':1/delta,
    'alpha_F':alpha_F,'alpha_F²':alpha_F**2,
}

# compose op lambdas
OPS = {}
for name,c in consts.items():
    OPS[f'×{name}'] = lambda x,c=c: x*c
    OPS[f'÷{name}'] = lambda x,c=c: x/c
    OPS[f'+{name}'] = lambda x,c=c: x+c
    OPS[f'-{name}'] = lambda x,c=c: x-c
    OPS[f'1-{name}'] = lambda x,c=c: comp(c)  # constant alone

print(f'Loaded {len(OPS)} ops')


Loaded 90 ops


In [6]:

# === plateaux & exhaustive run ===
def tag_plateaus(frame: pd.DataFrame, eps: float):
    base = frame['rel_err'].min()
    out  = frame.copy()
    out['plateau'] = ((out['rel_err']-base)/eps).astype(int)
    return out

views = []
for eps in [1e-4,5e-6,1e-5]:
    tagged = tag_plateaus(df, eps)
    for op_name, op_fun in OPS.items():
        tmp = tagged.copy()
        tmp['rel_err'] = op_fun(tmp['rel_err'])
        tmp['ε'] = eps
        tmp['op'] = op_name
        views.append(tmp)
        print(f'ε={eps:g}, op={op_name} → first row rel_err={tmp.rel_err.iloc[0]:.6g}')

out = pd.concat(views, ignore_index=True)

# save
raw_file = 'plateaus_raw.xlsx'
ops_file = 'plateaus_ops.xlsx'
df.to_excel(raw_file, index=False)
out.to_excel(ops_file, index=False)
print(f'Wrote {raw_file} and {ops_file}')


ε=0.0001, op=×9 → first row rel_err=3.70577
ε=0.0001, op=÷9 → first row rel_err=0.0457503
ε=0.0001, op=+9 → first row rel_err=9.41175
ε=0.0001, op=-9 → first row rel_err=-8.58825
ε=0.0001, op=1-9 → first row rel_err=-8
ε=0.0001, op=×3 → first row rel_err=1.23526
ε=0.0001, op=÷3 → first row rel_err=0.137251
ε=0.0001, op=+3 → first row rel_err=3.41175
ε=0.0001, op=-3 → first row rel_err=-2.58825
ε=0.0001, op=1-3 → first row rel_err=-2
ε=0.0001, op=×6 → first row rel_err=2.47052
ε=0.0001, op=÷6 → first row rel_err=0.0686254
ε=0.0001, op=+6 → first row rel_err=6.41175
ε=0.0001, op=-6 → first row rel_err=-5.58825
ε=0.0001, op=1-6 → first row rel_err=-5
ε=0.0001, op=×phi → first row rel_err=0.66623
ε=0.0001, op=÷phi → first row rel_err=0.254477
ε=0.0001, op=+phi → first row rel_err=2.02979
ε=0.0001, op=-phi → first row rel_err=-1.20628
ε=0.0001, op=1-phi → first row rel_err=-0.618034
ε=0.0001, op=×phi½ → first row rel_err=0.523757
ε=0.0001, op=÷phi½ → first row rel_err=0.3237
ε=0.0001, op=+p